In [1]:
import requests
from bs4 import BeautifulSoup
import urllib.parse
import tkinter as tk
from tkinter import messagebox, scrolledtext

def get_lyrics(song_name, artist_name):
    base_url = "https://www.lyricsfreak.com"
    search_query = f"{artist_name} {song_name}".replace(" ", "+")
    search_url = f"{base_url}/search.php?q={urllib.parse.quote(search_query)}"

    headers = {"User-Agent": "Mozilla/5.0"}
    search_response = requests.get(search_url, headers=headers)
    soup = BeautifulSoup(search_response.text, "html.parser")

    first_result = soup.select_one("a.song")
    if not first_result:
        return "Song not found."

    lyrics_page_url = base_url + first_result.get("href")
    lyrics_response = requests.get(lyrics_page_url, headers=headers)
    lyrics_soup = BeautifulSoup(lyrics_response.text, "html.parser")

    lyrics_div = lyrics_soup.find("div", class_="lyrictxt js-lyrics js-share-text-content")
    if not lyrics_div:
        return "Lyrics not found."

    return lyrics_div.get_text(separator="\n").strip()

def fetch_and_display_lyrics():
    song = song_entry.get()
    artist = artist_entry.get()
    if not song or not artist:
        messagebox.showwarning("Input Error", "Please enter both song and artist name.")
        return
    lyrics = get_lyrics(song, artist)
    lyrics_box.delete(1.0, tk.END)
    lyrics_box.insert(tk.END, lyrics)

# Create GUI window
root = tk.Tk()
root.title("Lyrics Finder")

# Input fields
tk.Label(root, text="Song Name:").grid(row=0, column=0, padx=5, pady=5, sticky="e")
song_entry = tk.Entry(root, width=40)
song_entry.grid(row=0, column=1, padx=5, pady=5)

tk.Label(root, text="Artist Name:").grid(row=1, column=0, padx=5, pady=5, sticky="e")
artist_entry = tk.Entry(root, width=40)
artist_entry.grid(row=1, column=1, padx=5, pady=5)

tk.Button(root, text="Get Lyrics", command=fetch_and_display_lyrics).grid(row=2, column=0, columnspan=2, pady=10)

# Scrolled text box for lyrics
lyrics_box = scrolledtext.ScrolledText(root, width=60, height=20, wrap=tk.WORD)
lyrics_box.grid(row=3, column=0, columnspan=2, padx=10, pady=10)

root.mainloop()